In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
stats = pd.read_csv('NBAstats.csv')
salary = pd.read_csv('NBASalary.csv')

In [9]:
stats = stats.rename(columns={"Unnamed: 0":"Rank"})


In [10]:
stats.head()

,Rank,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Steven Adams\adamsst01,C,26,OKC,63,63,1680,6.1,10.2,...,0.582,4.4,8.1,12.5,3.1,1.1,1.4,2.0,2.6,14.7
1,2,Bam Adebayo\adebaba01,PF,22,MIA,72,72,2417,6.6,11.8,...,0.691,2.6,8.3,10.9,5.5,1.2,1.4,3.0,2.7,17.1
2,3,LaMarcus Aldridge\aldrila01,C,34,SAS,53,53,1754,8.0,16.3,...,0.827,2.1,5.9,8.0,2.6,0.7,1.8,1.5,2.6,20.5
3,4,Kyle Alexander\alexaky01,PF,23,MIA,2,0,13,2.8,5.5,...,NaN,5.5,2.8,8.3,0.0,0.0,0.0,2.8,2.8,5.5
4,5,Nickeil Alexander-Walker\alexani01,SG,21,NOP,47,1,591,6.0,16.2,...,0.676,0.5,4.6,5.1,5.4,1.0,0.5,3.3,3.5,16.3


In [61]:
# Only using 2019-2020
salary = salary.drop(columns = ['Signed Using','Guaranteed','Rk'])
salary.head()

,Player,Tm,2019-20
0,Stephen Curry\curryst01,GSW,"$40,231,758"
1,Chris Paul\paulch01,OKC,"$38,506,482"
2,Russell Westbrook\westbru01,HOU,"$38,178,000"
3,John Wall\walljo01,WAS,"$37,800,000"
4,James Harden\hardeja01,HOU,"$37,800,000"


In [62]:
joint = stats.merge(salary,right_on = "Player",left_on = "Player")
joint

,Rank,Player,Pos,Age,Tm_x,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm_y,2019-20
0,1,Steven Adams\adamsst01,C,26,OKC,63,63,1680,6.1,10.2,...,8.1,12.5,3.1,1.1,1.4,2.0,2.6,14.7,OKC,"$25,842,697"
1,2,Bam Adebayo\adebaba01,PF,22,MIA,72,72,2417,6.6,11.8,...,8.3,10.9,5.5,1.2,1.4,3.0,2.7,17.1,MIA,"$3,454,080"
2,3,LaMarcus Aldridge\aldrila01,C,34,SAS,53,53,1754,8.0,16.3,...,5.9,8.0,2.6,0.7,1.8,1.5,2.6,20.5,SAS,"$26,000,000"
3,5,Nickeil Alexander-Walker\alexani01,SG,21,NOP,47,1,591,6.0,16.2,...,4.6,5.1,5.4,1.0,0.5,3.3,3.5,16.3,NOP,"$2,964,840"
4,6,Grayson Allen\allengr01,SG,24,MEM,38,0,718,5.9,12.6,...,3.9,4.3,2.6,0.5,0.1,1.7,2.7,16.5,MEM,"$2,429,400"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,524,Thaddeus Young\youngth01,PF,31,CHI,64,16,1591,6.1,13.6,...,5.0,7.1,2.6,2.1,0.5,2.3,3.0,14.9,CHI,"$12,900,000"
645,525,Trae Young\youngtr01,PG,21,ATL,60,60,2120,9.3,21.2,...,3.8,4.3,9.5,1.1,0.1,4.9,1.8,30.2,ATL,"$6,273,000"
646,526,Cody Zeller\zelleco01,C,27,CHO,58,39,1341,6.7,12.9,...,6.7,11.0,2.4,1.1,0.7,2.0,3.8,17.2,CHO,"$14,471,910"
647,527,Tyler Zeller\zellety01,C,30,SAS,2,0,4,9.0,36.0,...,9.0,36.0,0.0,0.0,0.0,0.0,0.0,18.0,SAS,"$245,686"


In [63]:
joint1 = joint.dropna(subset = ["2019-20","PTS"])
joint1

,Rank,Player,Pos,Age,Tm_x,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Tm_y,2019-20
0,1,Steven Adams\adamsst01,C,26,OKC,63,63,1680,6.1,10.2,...,8.1,12.5,3.1,1.1,1.4,2.0,2.6,14.7,OKC,"$25,842,697"
1,2,Bam Adebayo\adebaba01,PF,22,MIA,72,72,2417,6.6,11.8,...,8.3,10.9,5.5,1.2,1.4,3.0,2.7,17.1,MIA,"$3,454,080"
2,3,LaMarcus Aldridge\aldrila01,C,34,SAS,53,53,1754,8.0,16.3,...,5.9,8.0,2.6,0.7,1.8,1.5,2.6,20.5,SAS,"$26,000,000"
3,5,Nickeil Alexander-Walker\alexani01,SG,21,NOP,47,1,591,6.0,16.2,...,4.6,5.1,5.4,1.0,0.5,3.3,3.5,16.3,NOP,"$2,964,840"
4,6,Grayson Allen\allengr01,SG,24,MEM,38,0,718,5.9,12.6,...,3.9,4.3,2.6,0.5,0.1,1.7,2.7,16.5,MEM,"$2,429,400"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,524,Thaddeus Young\youngth01,PF,31,CHI,64,16,1591,6.1,13.6,...,5.0,7.1,2.6,2.1,0.5,2.3,3.0,14.9,CHI,"$12,900,000"
645,525,Trae Young\youngtr01,PG,21,ATL,60,60,2120,9.3,21.2,...,3.8,4.3,9.5,1.1,0.1,4.9,1.8,30.2,ATL,"$6,273,000"
646,526,Cody Zeller\zelleco01,C,27,CHO,58,39,1341,6.7,12.9,...,6.7,11.0,2.4,1.1,0.7,2.0,3.8,17.2,CHO,"$14,471,910"
647,527,Tyler Zeller\zellety01,C,30,SAS,2,0,4,9.0,36.0,...,9.0,36.0,0.0,0.0,0.0,0.0,0.0,18.0,SAS,"$245,686"


In [64]:
joint1["REBS"] = joint1["ORB"] + joint1["DRB"]
joint1["REBS"]

0      12.5
1      10.9
2       8.0
3       5.1
4       4.3
       ... 
644     7.1
645     4.3
646    11.0
647    36.0
648    14.7
Name: REBS, Length: 649, dtype: float64

In [65]:
joint1.drop(['Pos','Age','Tm_x','Tm_y'], axis = 1)

,Rank,Player,G,GS,MP,FG,FGA,FG%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2019-20,REBS
0,1,Steven Adams\adamsst01,63,63,1680,6.1,10.2,0.592,0.0,0.1,...,8.1,12.5,3.1,1.1,1.4,2.0,2.6,14.7,"$25,842,697",12.5
1,2,Bam Adebayo\adebaba01,72,72,2417,6.6,11.8,0.557,0.0,0.2,...,8.3,10.9,5.5,1.2,1.4,3.0,2.7,17.1,"$3,454,080",10.9
2,3,LaMarcus Aldridge\aldrila01,53,53,1754,8.0,16.3,0.493,1.3,3.2,...,5.9,8.0,2.6,0.7,1.8,1.5,2.6,20.5,"$26,000,000",8.0
3,5,Nickeil Alexander-Walker\alexani01,47,1,591,6.0,16.2,0.368,2.8,8.1,...,4.6,5.1,5.4,1.0,0.5,3.3,3.5,16.3,"$2,964,840",5.1
4,6,Grayson Allen\allengr01,38,0,718,5.9,12.6,0.466,2.9,7.1,...,3.9,4.3,2.6,0.5,0.1,1.7,2.7,16.5,"$2,429,400",4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,524,Thaddeus Young\youngth01,64,16,1591,6.1,13.6,0.448,1.8,5.0,...,5.0,7.1,2.6,2.1,0.5,2.3,3.0,14.9,"$12,900,000",7.1
645,525,Trae Young\youngtr01,60,60,2120,9.3,21.2,0.437,3.5,9.6,...,3.8,4.3,9.5,1.1,0.1,4.9,1.8,30.2,"$6,273,000",4.3
646,526,Cody Zeller\zelleco01,58,39,1341,6.7,12.9,0.524,0.5,2.0,...,6.7,11.0,2.4,1.1,0.7,2.0,3.8,17.2,"$14,471,910",11.0
647,527,Tyler Zeller\zellety01,2,0,4,9.0,36.0,0.250,0.0,0.0,...,9.0,36.0,0.0,0.0,0.0,0.0,0.0,18.0,"$245,686",36.0
